In [8]:
import os,requests,zlib,json
import pandas as pd
from pprint import pprint
from py2neo import Graph, NodeMatcher

In [9]:
## https://maayanlab.cloud/L1000FWD/download_page
## GOTO and download these csv & json files.
with open("L1000Download/CD_signatures_binary_42809.json") as f:
  cdData = json.load(f)
len(cdData) # ~43,000

42809

In [57]:
cdMeta = pd.read_csv('L1000Download/CD_signature_metadata.csv', low_memory=False)
drugMeta = pd.read_csv('L1000Download/Drugs_metadata.csv', low_memory=False)
cdMeta

,sig_id,SCS_centered_by_batch,batch,cell_id,mean_cosine_dist_centered_by_batch,pert_desc,pert_dose,pert_id,pert_time
0,AML001_CD34_6H:BRD-K43389675:10,0.0000,AML001_CD34_6H,CD34,0.853471,DAUNORUBICIN,10.00000,BRD-K43389675,6.0
1,AML001_PC3_6H:BRD-A19037878:0.37037,0.0000,AML001_PC3_6H,PC3,0.505995,TRICHOSTATIN_A,0.37037,BRD-A19037878,6.0
2,AML001_PC3_6H:BRD-A19037878:1.11111,0.0000,AML001_PC3_6H,PC3,0.676204,TRICHOSTATIN_A,1.11111,BRD-A19037878,6.0
3,AML001_PC3_6H:BRD-A19037878:10,0.0000,AML001_PC3_6H,PC3,0.747633,TRICHOSTATIN_A,10.00000,BRD-A19037878,6.0
4,AML001_PC3_6H:BRD-A19037878:3.33333,0.0000,AML001_PC3_6H,PC3,0.659851,TRICHOSTATIN_A,3.33333,BRD-A19037878,6.0
...,...,...,...,...,...,...,...,...,...
42804,RAD001_PC3_6H:BRD-K81418486-001-18-6:10,0.0000,RAD001_PC3_6H,PC3,0.761462,VORINOSTAT,10.00000,BRD-K81418486,6.0
42805,RAD001_PC3_6H:BRD-K81418486-001-18-6:3.3333,0.0064,RAD001_PC3_6H,PC3,0.868293,VORINOSTAT,3.33330,BRD-K81418486,6.0
42806,RAD001_PC3_6H:BRD-K81418486:10,0.0000,RAD001_PC3_6H,PC3,0.572592,vorinostat,10.00000,BRD-K81418486,6.0
42807,RAD001_PC3_6H:BRD-K84937637-001-03-2:0.1235,0.0191,RAD001_PC3_6H,PC3,0.906621,RAPAMYCIN,0.12350,BRD-K84937637,6.0


In [79]:
# set up authenticated connection
graph = Graph("bolt://localhost:7687/L1000Testing", auth=("neo4j", "ngs4"))

In [80]:
#i=0   128
for i in range (101,151):
    print(i)
    ## Add LOOP
    #pprint(cdData[i]['sig_id'])
    tRow = cdMeta.loc[cdMeta['sig_id'] == cdData[i]['sig_id']]
    #print(tRow.iloc[0]['pert_id'])

    ## Drug/Compund
    dRow = drugMeta.loc[drugMeta['pert_id'] == tRow.iloc[0]['pert_id']]
    altNomStr = ""
    if not dRow['alt_name'].isnull().values.any():
        altNomStr = ", AlternateName: '%s'"%(dRow.iloc[0]['alt_name'])
    collectStr = ""
    if not dRow['pert_collection'].isnull().values.any():
        collectStr = ", Collection: '%s'"%(dRow.iloc[0]['pert_collection'])
    chemStr = ""
    if not dRow['pubchem_cid'].isnull().values.any():
        chemStr = ", PubChem: '%s'"%(str(dRow.iloc[0]['pubchem_cid']))

    dId=tRow.iloc[0]['pert_id']
    dDesc=tRow.iloc[0]['pert_desc']
    drug1 = "MERGE (d:Drug {Id: '%s', Description: '%s' %s})"%(dId,dDesc,altNomStr+collectStr+chemStr)
    #print(drug1)

    ## Dose
    s2 = "MERGE (f:Dose { Amount: %s})"%(tRow.iloc[0]['pert_dose'])
    #print(s2)

    ## Time/Observation
    s3 = "MERGE (t:Time { Amount: %s})"%(tRow.iloc[0]['pert_time'])
    #print(s3)

    ## Cellline/Biologic
    s4 = "MERGE (c:CellType {Batch: '%s', Id: '%s'})"%(tRow.iloc[0]['batch'],tRow.iloc[0]['cell_id'])
    #print(s4)

    ## Signature
    signt5 = "MERGE (s:Signature {Id: '%s', SCS: %s, ClusterDistance: %s})"%(
        tRow.iloc[0]['sig_id'],tRow.iloc[0]['SCS_centered_by_batch'],
        tRow.iloc[0]['mean_cosine_dist_centered_by_batch'])
    #print(signt)

    ## Define Connections / Relationships
    r1 = "MERGE (d)-[:PERTURBED]-(s)"
    r2 = "MERGE (f)-[:ASSOCIATED]-(s)"
    r3 = "MERGE (t)-[:ASSOCIATED]-(s)"
    r4 = "MERGE (c)-[:ASSOCIATED]-(s)"

    subStmt = "{} {} {} {} {} {} {} {} {}".format(drug1,s2,s3,s4,signt5,r1,r2,r3,r4)
    #print(subStmt)
    graph.run(subStmt)


    for uGn in cdData[i]['up_genes']:
        gn = "MERGE (g:Gene { Symbol: '%s'})"%(uGn)
        rg = "MERGE (s)-[:UPREGULATED]-(g)"
        subStmt = "{} {} {}".format(gn,signt5,rg)
        #print(subStmt)
        graph.run(subStmt)


    for dGn in cdData[i]['down_genes']:
        gn = "MERGE (g:Gene { Symbol: '%s'})"%(dGn)
        rg = "MERGE (s)-[:DOWNREGULATED]-(g)"
        subStmt = "{} {} {}".format(gn,signt5,rg)
        #print(subStmt)
        graph.run(subStmt)

    pprint(cdData[i]['sig_id'])
    #print("Uploaded:"+cdData[i]['sig_id'])  
    #pprint(cdData[i])


101
'CPC014_MCF7_6H:BRD-A75409952:10'
102
'CPC002_PC3_24H:BRD-K71726959-001-04-2:10'
103
'CPC005_A549_6H:BRD-K56115039-003-01-3:10'
104
'CPC018_HT29_6H:BRD-K98548675-001-02-6:10'
105
'CPD003_MCF7_6H:BRD-A43671941-003-07-7:10'
106
'CPC012_ASC_24H:BRD-K14550461-001-01-3:10'
107
'CPC020_MCF7_6H:BRD-K21680192-003-15-1:10'
108
'MUC.CP004_MCF7_24H:BRD-K49049886-001-03-8:40'
109
'CPC019_VCAP_24H:BRD-K88278225-001-01-9:10'
110
'CPC006_PC3_6H:BRD-A31107743-001-01-3:0.09'
111
'CPC013_VCAP_24H:BRD-A19037878:10'
112
'CPC008_VCAP_24H:BRD-K82768082-001-01-7:10'
113
'CPC013_SKB_24H:BRD-K06623064-001-01-9:10'
114
'CPC003_HCC515_24H:BRD-A31007383-001-01-0:10'
115
'CPC011_VCAP_6H:BRD-K92984783-003-04-0:10'
116
'PCLB002_HT29_24H:BRD-K81418486:1.11'
117
'CPC010_A375_6H:BRD-A71262238-001-06-8:10'
118
'CPC011_VCAP_24H:BRD-K60038276-001-05-8:10'
119
'ERG011_VCAP_24H:BRD-A19037878-001-01-5:0.08'
120
'CPC005_VCAP_6H:BRD-A82238138-001-01-7:10'
121
'CPC009_A549_24H:BRD-K95138506-019-01-8:10'
122
'CPC013_HA1E_6H:

ClientError: [Statement.SyntaxError] Invalid input 't': expected whitespace, '.', node labels, '[', '^', '*', '/', '%', '+', '-', "=~", IN, STARTS, ENDS, CONTAINS, IS, '=', '~', "<>", "!=", '<', '>', "<=", ">=", AND, XOR, OR, ',' or ']' (line 1, column 105 (offset: 104))
"MERGE (d:Drug {Id: 'BRD-K73125384', Description: 'Acetic acid N-(2-methoxy-phenyl)-N'-[3-oxo-3H-benzo[b]thiophen-(2Z)-ylidene]-hydrazide' , Collection: 'COMB', PubChem: '5401876'}) MERGE (f:Dose { Amount: 10.0}) MERGE (t:Time { Amount: 6.0}) MERGE (c:CellType {Batch: 'CPC010_HT29_6H', Id: 'HT29'}) MERGE (s:Signature {Id: 'CPC010_HT29_6H:BRD-K73125384-001-05-7:10', SCS: 0.0052, ClusterDistance: 0.959655949089}) MERGE (d)-[:PERTURBED]-(s) MERGE (f)-[:ASSOCIATED]-(s) MERGE (t)-[:ASSOCIATED]-(s) MERGE (c)-[:ASSOCIATED]-(s)"
                                                                                                         ^

In [78]:
pathways = pd.read_csv('Biocarta Pathways/gene_attribute_edges.txt', sep='\t')
pathways = pathways.drop(["source_desc","target_desc"], axis=1)
pathways = pathways.iloc[1:]
pathways.head(3)
for row in pathways.itertuples():
    s2 = "MERGE (g:Gene { Symbol: '%s'}) SET g.EntrezId='%s' "%(row.source, row.source_id)
    s3 = "MERGE (p:Pathway { Name: '%s', Id: '%s', Origin: 'Biocarta'})"%(row.target,row.target_id)
    r1 = "MERGE (g)-[:BELONGS_TO{weight:%s}]->(p)"%(row.weight)
    subStmt = "{} {} {}".format(s2,s3,r1)
    graph.run(subStmt)

pathways = pd.read_csv('HumanCyc Pathways/gene_attribute_edges.txt', sep='\t')
pathways = pathways.drop(["target_id","target_desc"], axis=1)
pathways = pathways.iloc[1:]
pathways.head(3)
for row in pathways.itertuples():
    s2 = "MERGE (g:Gene { Symbol: '%s'}) SET g.EntrezId='%s', g.UniprotACC='%s' "%(row.source, row.source_id, row.source_desc)
    s3 = "MERGE (p:Pathway { Name: '%s', Origin: 'HumanCyc'})"%(row.target.replace('&','').replace(';','').replace("'",''))
    r1 = "MERGE (g)-[:BELONGS_TO{weight:%s}]->(p)"%(row.weight)
    subStmt = "{} {} {}".format(s2,s3,r1)
    graph.run(subStmt)
    
pathways = pd.read_csv('KEGG Pathways/gene_attribute_edges.txt', sep='\t')
pathways = pathways.drop(["source_desc","target_id"], axis=1)
pathways = pathways.iloc[1:]
pathways.head(3)
for row in pathways.itertuples():
    s2 = "MERGE (g:Gene {Symbol: '%s'}) SET g.EntrezId='%s'"%(row.source, row.source_id)
    s3 = "MERGE (p:Pathway {Name: '%s', Id: '%s', Origin:'Kegg'})"%(row.target,row.target_desc)
    r1 = "MERGE (g)-[:BELONGS_TO{weight:%s}]->(p)"%(row.weight)
    subStmt = "{} {} {}".format(s2,s3,r1)
    graph.run(subStmt)